In [127]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

token = {
    '.' : 0, 
    ',' : 1, 
    '[' : 2, 
    ']' : 3, 
    '<' : 4, 
    '>' : 5, 
    '+' : 6, 
    '-' : 7,
    "START" : 8
    }

char = {
    0 : '.',
    1 : ',', 
    2 : '[', 
    3 : ']',  
    4 : '<',  
    5 : '>', 
    6 : '+',  
    7 : '-'
    # no START on purpose
    }

class BFgen(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, output_size, n_layers=2, batch_size=1):
        super(BFgen, self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.batch_size = batch_size
        
        self.encoder = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = nn.functional.softmax
        

    """
    forward
    Takes input_token and hidden memory state <- input to recursive layer
    returns output token and changed hidden memory state.
    """
    def forward(self, input_token):
        embeds = self.encoder(input_token)
        output, self.hidden = self.lstm(embeds, self.hidden)
        decoded = self.decoder(output)
        probs = self.softmax(decoded[-1])
        return probs
    
    def init_hidden_zero(self):
        self.hidden = (Variable(torch.zeros(self.n_layers, self.batch_size, self.hidden_size)),
                      Variable(torch.zeros(self.n_layers, self.batch_size, self.hidden_size)),)
    
    def init_hidden_normal(self, variance=0.01):
        means = torch.zeros(self.n_layers, self.batch_size, self.hidden_size)
        std = torch.Tensor([variance]*self.hidden_size*self.n_layers*self.batch_size).unsqueeze(0)
        self.hidden = (Variable(torch.normal(means, std)), Variable(torch.normal(means, std)))





In [128]:
def token_to_tensor(input_token):
    tensor = torch.zeros(1, token_num).long()
    tensor[0][token[input_token]] = 1
    return tensor

def pred(sample):
    return np.argsort(sample)[-1]

In [258]:
embedding_size = 10
hidden_size = 35
output_size = len(char.keys())
n_layers = 2
batch_size = 16
GAMMA = 0.99 # for exponential moving avarage constant from paper


token_num = len(token.keys())

In [146]:
def evaluate(model, predict_len=100, variance=0.01):
    input_token = "START"
    input_token = token_to_tensor(input_token)
    model.init_hidden_normal(variance=0.5)
    prediction = [""] * batch_size
    program_probs = np.ones((1, batch_size))
    
    batched_input = torch.zeros((batch_size, token_num)).long()
    batched_input = batched_input + input_token
    batched_input = Variable(batched_input.view(token_num, batch_size))
    
    for i in range(predict_len):
        output_probs = model.forward(batched_input)
        top_probs, next_tokens = torch.max(output_probs, 1)
        next_tokens = next_tokens.data.numpy()
            
        batched_input = torch.zeros((batch_size, token_num)).long()
        batched_input[np.arange(batch_size), next_tokens] = 1
        batched_input = Variable(batched_input.view(token_num, batch_size))
        
        program_probs *= top_probs.data.numpy()
        
        #prediction = map("".join,zip(prediction, char[]))
        for i in xrange(batch_size):
            prediction[i] += char[next_tokens[i]]
        
    return prediction, program_probs

In [147]:
model = BFgen(token_num, embedding_size, hidden_size, output_size, n_layers, batch_size)

In [164]:
progs, probs = evaluate(model, 5, 10.0)

In [165]:
print(progs)

[']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]', ']]]]]']


In [277]:
def objective_PG(model, reward_f, predict_len = 100, N = 1000):
    objective = Variable(torch.FloatTensor([1]))
    baseline = 0
    for i in range(0, N, model.batch_size):
        input_token = "START"
        input_token = token_to_tensor(input_token)
        model.init_hidden_normal(variance=0.5)
        prediction = [""] * model.batch_size
        policy_probs = Variable(torch.zeros(model.batch_size).float())

        batched_input = torch.zeros((batch_size, token_num)).long()
        batched_input = batched_input + input_token
        batched_input = Variable(batched_input.view(token_num, model.batch_size))

        for i in range(predict_len):
            output_probs = model.forward(batched_input)
            top_probs, next_tokens = torch.max(output_probs, 1)
            next_tokens = next_tokens.data.numpy()
            
            batched_input = torch.zeros((batch_size, token_num)).long()
            batched_input[np.arange(batch_size), next_tokens] = 1
            batched_input = Variable(batched_input.view(token_num, model.batch_size))
            
            # accumulate objective
            torch.log(top_probs)
            policy_probs += top_probs
            
            # save chars
            for i in xrange(batch_size):
                prediction[i] += char[next_tokens[i]]
        
        # calculate rewards
        rewards = map(reward_f, prediction)
        # exponential moving avarage
        print(rewards)
        for i, reward in enumerate(rewards):
            baseline = reward * GAMMA + (1 - GAMMA) * baseline
            rewards[i] = reward - baseline
            
        rewards = Variable(torch.FloatTensor(rewards))
        objective += (rewards * policy_probs).sum()
    return objective / N
        

In [278]:
print(objective_PG(model, lambda x : 1, N=16))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.010000000000000009, 9.999999999998899e-05, 1.0000000000287557e-06, 1.0000000050247593e-08, 1.000000082740371e-10, 9.999778782798785e-13, 9.992007221626409e-15, 1.1102230246251565e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Variable containing:
1.00000e-02 *
  7.1477
[torch.FloatTensor of size 1]

